In [1]:
ls *csv

acp_enrollment_tribal_areas_date.csv
acp_enrollment_tribal_areas_date_area.csv
dl_acp_dash_zip_county.csv
geocorr2022_county_tribal_areas_hh_20.csv
geocorr2022_county_tribal_areas_larea_20.csv
ps_acp_eligible_county.csv
ps_acp_participating_county.csv


In [2]:
import pandas as pd

## Paricipating/ Enrolled

In [3]:
infile='ps_acp_participating_county.csv'
dp=pd.read_csv(infile)
dp.drop(columns=['state_id','county_id'],inplace=True)
dp.rename(columns={'concat':'county_id'},inplace=True)
print(dp.info())
dp.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36032 entries, 0 to 36031
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   county_id     36032 non-null  object 
 1   date          36032 non-null  object 
 2   acp_enrolled  35995 non-null  float64
dtypes: float64(1), object(2)
memory usage: 844.6+ KB
None


,county_id,date,acp_enrolled
0,36001,2022-09-30,19077.0
1,36001,2022-10-31,19777.0
2,36001,2022-11-30,20355.0
3,30001,2022-04-30,138.0
4,30001,2022-01-31,113.0


## Eligible

In [4]:
infile='ps_acp_eligible_county.csv'
dtype_dic={'county_id':str}
de=pd.read_csv(infile,dtype=dtype_dic)
de.drop(columns=['state_id','households_acp_eligible_unconnected'],inplace=True)
print(de.info())
print('========= nunique')
print(de.nunique())
de.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 2 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   county_id                3142 non-null   object 
 1   households_acp_eligible  3142 non-null   float64
dtypes: float64(1), object(1)
memory usage: 49.2+ KB
None
========= nunique
county_id                  3142
households_acp_eligible    3136
dtype: int64


,county_id,households_acp_eligible
0,01001,8660.4698
1,01003,32727.0000
2,01005,5835.5520
3,01007,4162.7172
4,01009,9412.4980


## ACP rates

In [5]:
dr=dp.merge(de,on='county_id',how='inner')
print(dr.info())
print('========= nunique')
print(dr.nunique())
dr.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34441 entries, 0 to 34440
Data columns (total 4 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   county_id                34441 non-null  object 
 1   date                     34441 non-null  object 
 2   acp_enrolled             34432 non-null  float64
 3   households_acp_eligible  34441 non-null  float64
dtypes: float64(2), object(2)
memory usage: 1.3+ MB
None
========= nunique
county_id                  3137
date                         11
acp_enrolled               8587
households_acp_eligible    3131
dtype: int64


,county_id,date,acp_enrolled,households_acp_eligible
0,36001,2022-09-30,19077.0,48871.0
1,36001,2022-10-31,19777.0,48871.0
2,36001,2022-11-30,20355.0,48871.0
3,36001,2022-01-31,11372.0,48871.0
4,36001,2022-04-30,14345.0,48871.0


In [6]:
#dr['adoption_rate']=dr['acp_enrolled']/dr['households_acp_eligible']
#dr[dr['county_id'].isin(['36001','36003'])].sort_values(by=['county_id','date'])

### Crosswalk

In [17]:
infile='geocorr2022_county_tribal_areas_hh_20.csv'
dtype_dic={'County code':str}
dh=pd.read_csv(infile,skiprows=1,encoding = 'latin1',dtype=dtype_dic)
print(dh.info())
dh.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4123 entries, 0 to 4122
Data columns (total 7 columns):
 #   Column                                                          Non-Null Count  Dtype  
---  ------                                                          --------------  -----  
 0   County code                                                     4123 non-null   object 
 1   American Indian / Alaska Native / Native Hawaiian areas (2021)  4123 non-null   object 
 2   County name                                                     4123 non-null   object 
 3   AIANNH area name                                                4123 non-null   object 
 4   Total housing units (2020 Census)                               4123 non-null   int64  
 5   aiannh-to-county allocation factor                              4123 non-null   float64
 6   county-to-aiannh allocation factor                              4123 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 225

,County code,American Indian / Alaska Native / Native Hawaiian areas (2021),County name,AIANNH area name,Total housing units (2020 Census),aiannh-to-county allocation factor,county-to-aiannh allocation factor
0,01001,,Autauga AL,,24350,0.0002,1.0
1,01003,,Baldwin AL,,124148,0.0009,1.0
2,01005,,Barbour AL,,11618,0.0001,1.0
3,01007,,Bibb AL,,9002,0.0001,1.0
4,01009,,Blount AL,,24622,0.0002,1.0


In [18]:
#Cleaning for empty tribal areas code/name

dh=dh[~dh['American Indian / Alaska Native / Native Hawaiian areas (2021)'].isin([' '])].sort_values(by=['American Indian / Alaska Native / Native Hawaiian areas (2021)'])
print(dh.info())
dh.reset_index(drop=True,inplace=True)


names={'County code':'county_id',
       'American Indian / Alaska Native / Native Hawaiian areas (2021)':'aiannh','county-to-aiannh allocation factor':'afact'}
dh.rename(columns=names,inplace=True)
cols=['county_id','aiannh','afact']
dh=dh[cols]


print(dh.info())
print('========= nunique')
print(dh.nunique())
dh.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 973 entries, 2436 to 3109
Data columns (total 7 columns):
 #   Column                                                          Non-Null Count  Dtype  
---  ------                                                          --------------  -----  
 0   County code                                                     973 non-null    object 
 1   American Indian / Alaska Native / Native Hawaiian areas (2021)  973 non-null    object 
 2   County name                                                     973 non-null    object 
 3   AIANNH area name                                                973 non-null    object 
 4   Total housing units (2020 Census)                               973 non-null    int64  
 5   aiannh-to-county allocation factor                              973 non-null    float64
 6   county-to-aiannh allocation factor                              973 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6

,county_id,aiannh,afact
0,35006,0010,0.1035
1,35053,0010,0.0004
2,06065,0020,0.0297
3,48373,0050,0.0098
4,36009,0080,0.0808


### ACP ~ tribal

In [20]:
dx=dr.merge(dh,on='county_id',how='inner')
print(dx.info())
print('========= nunique')
print(dx.nunique())
dx.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10354 entries, 0 to 10353
Data columns (total 6 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   county_id                10354 non-null  object 
 1   date                     10354 non-null  object 
 2   acp_enrolled             10354 non-null  float64
 3   households_acp_eligible  10354 non-null  float64
 4   aiannh                   10354 non-null  object 
 5   afact                    10354 non-null  float64
dtypes: float64(3), object(3)
memory usage: 566.2+ KB
None
========= nunique
county_id                   471
date                         11
acp_enrolled               3094
households_acp_eligible     471
aiannh                      636
afact                       473
dtype: int64


,county_id,date,acp_enrolled,households_acp_eligible,aiannh,afact
0,04001,2022-06-30,13340.0,13289.1965,1140,0.0062
1,04001,2022-06-30,13340.0,13289.1965,2430,0.6164
2,04001,2022-06-30,13340.0,13289.1965,4785,0.0000
3,04001,2022-07-31,13004.0,13289.1965,1140,0.0062
4,04001,2022-07-31,13004.0,13289.1965,2430,0.6164


In [23]:
dx['acp_enrolled_aiannh']=dx['acp_enrolled']*dx['afact']
dx['households_acp_eligible_aiannh']=dx['households_acp_eligible']*dx['afact']

In [24]:
cols=['date','aiannh','acp_enrolled_aiannh','households_acp_eligible_aiannh']
dx[cols]

,date,aiannh,acp_enrolled_aiannh,households_acp_eligible_aiannh
0,2022-06-30,1140,82.7080,82.393018
1,2022-06-30,2430,8222.7760,8191.460723
2,2022-06-30,4785,0.0000,0.000000
3,2022-07-31,1140,80.6248,82.393018
4,2022-07-31,2430,8015.6656,8191.460723
...,...,...,...,...
10349,2022-03-31,3980,5.1024,21.210317
10350,2022-03-31,4560,0.4252,1.767526
10351,2022-02-28,0510,6.6674,30.047949
10352,2022-02-28,3980,4.7064,21.210317


## ACP monthly numbers for tribal areas

In [25]:
x=dx[['date','acp_enrolled_aiannh','households_acp_eligible_aiannh']].groupby('date').sum()
x['acp_adoption_rate_aiannh']=x['acp_enrolled_aiannh']/x['households_acp_eligible_aiannh']
x.reset_index(inplace=True)
x

,date,acp_enrolled_aiannh,households_acp_eligible_aiannh,acp_adoption_rate_aiannh
0,2022-01-31,211701.6482,909392.821899,0.232795
1,2022-02-28,222523.2911,909533.262717,0.244657
2,2022-03-31,238238.6823,909533.262717,0.261935
3,2022-04-30,246638.7164,909533.262717,0.271171
4,2022-05-31,257069.1311,909533.262717,0.282639
5,2022-06-30,269343.2159,909533.262717,0.296133
6,2022-07-31,278991.3593,909533.262717,0.306741
7,2022-08-31,289570.0092,909534.414614,0.318372
8,2022-09-30,302574.6437,909534.414614,0.332670
9,2022-10-31,316411.4506,909534.414614,0.347883


In [27]:
outfile='acp_enrollment_tribal_areas_date.csv'
x.to_csv(outfile,index=False)

## ACP numbers by date and tribal area number

In [28]:
y=dx[['date','aiannh','acp_enrolled_aiannh','households_acp_eligible_aiannh']].groupby(['date','aiannh']).sum()
y=y.reset_index()
y['acp_adoption_rate']=y['acp_enrolled_aiannh']/y['households_acp_eligible_aiannh']
y.sort_values(by=['aiannh','date'],inplace=True)
y

,date,aiannh,acp_enrolled_aiannh,households_acp_eligible_aiannh,acp_adoption_rate
0,2022-01-31,0010,178.4227,680.758486,0.262094
635,2022-02-28,0010,187.6546,680.758486,0.275655
1271,2022-03-31,0010,198.8614,680.758486,0.292117
1907,2022-04-30,0010,202.5958,680.758486,0.297603
2543,2022-05-31,0010,205.4794,680.758486,0.301839
...,...,...,...,...,...
4450,2022-07-31,9975,160.5552,977.368014,0.164273
5086,2022-08-31,9975,175.0968,977.368014,0.179151
5722,2022-09-30,9975,186.0196,977.368014,0.190327
6358,2022-10-31,9975,199.0672,977.368014,0.203677


In [29]:
outfile='acp_enrollment_tribal_areas_date_area.csv'
y.to_csv(outfile,index=False)

In [35]:
ls acp*

acp_adoption_tribal_area_v0.ipynb
acp_enrollment_tribal_areas_date.csv
acp_enrollment_tribal_areas_date_area.csv
acp_enrollment_tribal_areas_date_area.xlsx


In [31]:
outfile='acp_enrollment_tribal_areas_date_area.xlsx'
y.to_excel(outfile,index=False)